In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
!ls

Untitled.ipynb    seg_pred          seg_train
intel-image.ipynb seg_test


In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        'seg_train/seg_train',
        target_size=(50, 50),
        batch_size=64,
        class_mode='binary') 

Found 14034 images belonging to 6 classes.


In [5]:
test_generator = test_datagen.flow_from_directory(
    'seg_test/seg_test',
    target_size=(50,50),
    batch_size=64,
    class_mode='binary'
)

Found 3000 images belonging to 6 classes.


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,MaxPool2D,Flatten,Dropout
from tensorflow.keras import optimizers

In [7]:
model = Sequential()

model.add(Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(50,50,3)))
model.add(Conv2D(180,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(3,3))
model.add(Conv2D(180,kernel_size=(3,3),activation='relu'))
#model.add(Conv2D(140,kernel_size=(3,3),activation='relu'))
#model.add(Conv2D(100,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(50,kernel_size=(3,3),activation='relu'))
model.add(MaxPool2D(5,5))
model.add(Flatten())
model.add(Dense(180,activation='relu'))
#model.add(Dense(100,activation='relu'))
model.add(Dense(50,activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(6,activation='softmax'))

model.compile(optimizer=optimizers.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 200)       5600      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 180)       324180    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 180)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 180)       291780    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 50)        81050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 50)          0         
_________________________________________________________________
flatten (Flatten)            (None, 200)               0

In [ ]:
model.fit_generator(
        train_generator,
        #steps_per_epoch=2000,
        epochs=5,
        validation_data=test_generator,
        validation_steps=800)